In [5]:
import os
%pwd

'/Users/shashwatkushwaha/Desktop/ML/TextSummarizer/research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'/Users/shashwatkushwaha/Desktop/ML/TextSummarizer'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_path: Path



In [9]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.params = read_yaml(params_filepath)
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_path = config.metric_file_path
        )

        return model_evaluation_config

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from datasets import load_dataset,load_from_disk
import pandas as pd
from tqdm import tqdm
import evaluate 

In [12]:
class ModelEvaluation:
    def __init__(self,config:ArithmeticError):
        self.config = config

    def generate_batch_sized_chunks(self,list_of_elements,batch_size):
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]

    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,batch_size=16,column_text='article',column_summary = 'highlights'):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        for article_batch,target_batch in tqdm(
            zip(article_batches,target_batches),total = len(article_batches)
        ):
            inputs = tokenizer(article_batch,max_length=1024,truncation=True,padding='max_length',return_tensors='pt')
            summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                    attention_mask = inputs['attention_mask'].to(device),
                                    length_penalty = 0.8,num_beams=1)
            decoded_summaries = [tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]
            decoded_summaries = [d.replace(""," ") for d in decoded_summaries]
            metric.add_batch(predictions = decoded_summaries,references = target_batch)

        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ['rouge1',"rouge2","rougeL",'rougeLsum']
        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10],rouge_metric,model_pegasus,tokenizer,batch_size=1,column_text='dialogue',column_summary='summary'
        )
        rouge_dict = dict((rn,score[rn]) for rn in rouge_names)
        df = pd.DataFrame(rouge_dict,index=[f'pegasus'])
        df.to_csv(self.config.metric_file_path, index=False)

In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2025-12-24 00:23:31,922 : INFO : common : yaml file : <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-12-24 00:23:31,926 : INFO : common : yaml file : <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-12-24 00:23:31,929 : INFO : common : Created Directory at artifacts]
[2025-12-24 00:23:31,930 : INFO : common : Created Directory at artifacts/model_evaluation]


100%|██████████| 10/10 [00:25<00:00,  2.54s/it]

[2025-12-24 00:24:16,017 : INFO : rouge_scorer : Using default tokenizer.]
